In [1]:
from ase.build import bulk, make_supercell
import numpy as np
import HopDec.Dimer as dimer
import HopDec.Minimize as min
import HopDec.NEB as neb
import HopDec.Redecorate as redec
from HopDec.Input import getParams
from HopDec.ASE import ASE
from HopDec.Lammps import LammpsInterface

In [2]:
# Generate simple ASE structure

CuPrim = bulk('Cu', 'fcc', a=3.6, cubic=True)
CuPrim.cell
CuSuperCell = make_supercell(CuPrim, np.array([[5,0,0],[0,5,0],[0,0,5]]))

In [3]:
# Create Vacancy in supercell

del CuSuperCell[249]

In [4]:
# Initialize HopDec and transform ASE structure to State object

params = getParams()
ase = ASE(params)
state = ase.toState(CuSuperCell)

In [5]:
# mininize State in-place

lmp = LammpsInterface(params)
min.main(state,lmp,params)

In [6]:
# run dimer to find new state

d = dimer.main(state,params,None, pushSaddle = True)

[17/02/24, 16:23:28]:   Lammps >> Minimizing. F Tol: 1e-05.
[17/02/24, 16:23:28]: Lammps >> Completed Minimization, E: -1764.8919 eV
[17/02/24, 16:23:28]: Dimer >> Found saddle point, 0.7921867056008978 eV above inital position
[17/02/24, 16:23:28]: Dimer >> Pushing saddle point into new minima
[17/02/24, 16:23:28]:   Lammps >> Minimizing. F Tol: 1e-05.
[17/02/24, 16:23:29]: Lammps >> Completed Minimization, E: -1764.8919 eV


In [7]:
# run NEB for barrier
newState = d.newMinimaState
n = neb.main(state, newState, params)

In [8]:
# return results through NEBResults object

print(n.transitions[0].forwardBarrier)
print(n.transitions[0].reverseBarrier)
print(n.transitions[0].dE)

0.7411
0.7411
0.0


In [9]:
# pass new structure back to ASE object

newState.toASE(params)

Atoms(symbols='Cu499', pbc=True, cell=[18.0, 18.0, 18.0])

In [10]:
# Redecoration campaign
params.nDecorations = 10
redecResults = redec.main(state, newState, params)


[17/02/24, 16:23:34]: Redecorate >> Starting NEB Redecoration Campaign
[17/02/24, 16:23:34]:   Redecorate >> rank: 0: Redecoration: 1
[17/02/24, 16:23:37]:   Redecorate >> rank: 0: Redecoration: 2
[17/02/24, 16:23:40]:   Redecorate >> rank: 0: Redecoration: 3
[17/02/24, 16:23:43]:   Redecorate >> rank: 0: Redecoration: 4
[17/02/24, 16:23:46]:   Redecorate >> rank: 0: Redecoration: 5
[17/02/24, 16:23:48]:   Redecorate >> rank: 0: Redecoration: 6
[17/02/24, 16:23:51]:   Redecorate >> rank: 0: Redecoration: 7
[17/02/24, 16:23:54]:   Redecorate >> rank: 0: Redecoration: 8
[17/02/24, 16:23:57]:   Redecorate >> rank: 0: Redecoration: 9
[17/02/24, 16:24:00]:   Redecorate >> rank: 0: Redecoration: 10


In [11]:
redecResults.summarize()

[17/02/24, 16:24:03]: Redecorate >> Summary:
	Connection 1:
		Transition 1:
			transition.KRA = 0.8753
			transition.dE = 0.2462
	Connection 2:
		Transition 1:
			transition.KRA = 0.8652
			transition.dE = 0.4501
	Connection 3:
		Transition 1:
			transition.KRA = 0.7931
			transition.dE = 0.0662
	Connection 4:
		Transition 1:
			transition.KRA = 0.9897
			transition.dE = -0.1585
	Connection 5:
		Transition 1:
			transition.KRA = 0.7356
			transition.dE = 0.1463
	Connection 6:
		Transition 1:
			transition.KRA = 1.234
			transition.dE = 0.1195
	Connection 7:
		Transition 1:
			transition.KRA = 0.907
			transition.dE = 0.0717
	Connection 8:
		Transition 1:
			transition.KRA = 0.7482
			transition.dE = 0.0321
	Connection 9:
		Transition 1:
			transition.KRA = 0.7475
			transition.dE = -0.1578
	Connection 10:
		Transition 1:
			transition.KRA = 1.0304
			transition.dE = 0.2424
